In [26]:
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
import numpy as np

In [27]:
train_path = "../data/origin/train.csv"
building_path = "../data/origin/building_info.csv"
train = pd.read_csv(train_path, encoding='utf-8')
building = pd.read_csv(building_path, encoding='utf-8')

ko2en_dict = {
 '건물번호': 'b_num',
 '일시': 'date',
 '기온(°C)': 'tmp',
 '강수량(mm)': 'rain',
 '풍속(m/s)': 'wind',
 '습도(%)': 'hum',
 '일조(hr)': 'sunshine',
 '일사(MJ/m2)': 'solar',
 '전력소비량(kWh)': 'power_consumption',
 '건물유형': 'b_type',
 '연면적(m2)': 'total_area',
 '냉방면적(m2)': 'cooling_area',
 '태양광용량(kW)': 'solar_capacity',
 'ESS저장용량(kWh)': 'ess_capacity',
 'PCS용량(kW)': 'pcs_capacity',
}

def rename_dataframe_columns(df, mapping_dict):
    return df.rename(columns=mapping_dict).copy()

train_df = rename_dataframe_columns(train, ko2en_dict)
train_df['datetime'] = pd.to_datetime(train['일시'], format='%Y%m%d %H')

# 요일 컬럼 추가 (0=월, 6=일)
train_df['weekday'] = train_df['datetime'].dt.weekday  # 숫자 요일
train_df['month'] = train_df['datetime'].dt.month
train_df['day'] = train_df['datetime'].dt.day
train_df['time'] = train_df['datetime'].dt.hour

train_df = train_df.drop("date", axis=1)
building_info_df = rename_dataframe_columns(building, ko2en_dict)

merge_df = pd.merge(train_df, building_info_df, on='b_num', how='left')
display(merge_df.head())
print(list(merge_df.columns))

,num_date_time,b_num,tmp,rain,wind,hum,sunshine,solar,power_consumption,datetime,weekday,month,day,time,b_type,total_area,cooling_area,solar_capacity,ess_capacity,pcs_capacity
0,1_20240601 00,1,18.3,0.0,2.6,82.0,0.0,0.0,5794.80,2024-06-01 00:00:00,5,6,1,0,호텔,82912.71,77586.0,-,-,-
1,1_20240601 01,1,18.3,0.0,2.7,82.0,0.0,0.0,5591.85,2024-06-01 01:00:00,5,6,1,1,호텔,82912.71,77586.0,-,-,-
2,1_20240601 02,1,18.1,0.0,2.6,80.0,0.0,0.0,5338.17,2024-06-01 02:00:00,5,6,1,2,호텔,82912.71,77586.0,-,-,-
3,1_20240601 03,1,18.0,0.0,2.6,81.0,0.0,0.0,4554.42,2024-06-01 03:00:00,5,6,1,3,호텔,82912.71,77586.0,-,-,-
4,1_20240601 04,1,17.8,0.0,1.3,81.0,0.0,0.0,3602.25,2024-06-01 04:00:00,5,6,1,4,호텔,82912.71,77586.0,-,-,-


['num_date_time', 'b_num', 'tmp', 'rain', 'wind', 'hum', 'sunshine', 'solar', 'power_consumption', 'datetime', 'weekday', 'month', 'day', 'time', 'b_type', 'total_area', 'cooling_area', 'solar_capacity', 'ess_capacity', 'pcs_capacity']


In [28]:
def is_drop(df, col):
    '''col 전체가 결측치면 True 하나라도 참값이 있다면 False'''
    if df[col].isna().all():
        return True
    else:
        return False
    
def convert_day(df, threshold=0.018):
    '''주말의 평균 전력사용량과 평일의 전력사용량의 비율 차이가 0.018 이상 차이나면 범주화'''
    gb = df.groupby(['weekday']).mean(numeric_only=True)

    workday = (sum(gb['power_consumption'].iloc[0:5])/5)/sum(gb['power_consumption']) 
    holiday = (sum(gb['power_consumption'].iloc[5:])/2)/sum(gb['power_consumption'])
    if workday - holiday > threshold:
        return True
    else:
        return False
    
def minmax_scale(df: pd.DataFrame, exclude_cols):
    '''MinMax Scalering 적용'''
    target_cols = [i for i in df.columns if i not in exclude_cols]

    scaler = MinMaxScaler()
    df[target_cols] = scaler.fit_transform(df[target_cols])

    return df

In [ ]:

change_name = ['hotel', 'commercial', 'hospital', 'school', 'etc', 'apart', 'research', 'store', 'idc','public']
df_dict = {}

for i, tp in enumerate(merge_df['b_type'].unique()):
    exclude_list = ['time', 'power_consumption']
    drop_list = ['num_date_time', 'b_num', 'datetime', 'month', 'day', 'b_type', 'total_area']

    name = change_name[i]
    df_dict[name] = merge_df[merge_df['b_type'] == tp].reset_index(drop=True)

    for col in df_dict[name].columns:
        if is_drop(df_dict[name], col):
            drop_list.append(col)
        else:
            df_dict[name][col] = df_dict[name][col].replace("-", 0)

    if convert_day(df_dict[name]):
        df_dict[name]['weekday'] = df_dict[name]['weekday'].apply(lambda x: 0 if x < 5 else 1)
        exclude_list.append('weekday')
    else:
        drop_list.append('weekday')

    df_dict[name].drop(drop_list, axis=1, inplace=True)
    df_dict[name] = minmax_scale(df_dict[name], exclude_list)
    df_dict[name].to_csv(f"../data/processed/{name}_train.csv", encoding='utf-8',index=False)

C:\Users\gd980\AppData\Local\Temp\ipykernel_17972\775695525.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_dict[name][col] = df_dict[name][col].replace("-", 0)
C:\Users\gd980\AppData\Local\Temp\ipykernel_17972\775695525.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_dict[name][col] = df_dict[name][col].replace("-", 0)
C:\Users\gd980\AppData\Local\Temp\ipykernel_17972\775695525.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior,

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

def smape(y_true, y_pred):
    return 100 * np.mean(
        2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-9)
    )
smape_scorer = make_scorer(lambda y_true, y_pred: -smape(y_true, y_pred))

default_params = {
    "XGB": {
        "n_estimators": 500,
        "learning_rate": 0.1,
        "max_depth": 7,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "random_state": 42,
        "n_jobs": -1,
        "verbosity": 0  
    },
    "LGBM": {
        "n_estimators": 500,
        "learning_rate": 0.1,
        "max_depth": -1,
        "num_leaves": 63,
        "subsample": 0.8,
        "random_state": 42,
        "n_jobs": -1,
        "verbose": -1   
    },
    "RF": {
        "n_estimators": 500,
        "max_depth": None,
        "min_samples_split": 2,
        "min_samples_leaf": 1,
        "max_features": "sqrt",
        "random_state": 42,
        "n_jobs": -1
    }
}

def train_and_select_without_tuning(models, X_train, y_train):
    """기본 파라미터로 각 모델을 5-Fold CV로 평가하고, SMAPE 최소 모델 선택"""
    results = {}
    cv = KFold(n_splits=5, shuffle=True, random_state=42)

    for name, bundle in models.items():
        est_cls = type(bundle["model"])
        params = default_params.get(name, {})
        est = est_cls(**params)
        scores = cross_val_score(est, X_train, y_train,
                                 scoring=smape_scorer, cv=cv, n_jobs=-1)
        mean_smape = -float(np.mean(scores))
        results[name] = {
            "cv_smape": mean_smape,
            "params": params,
            "color": bundle.get("color", "Black"),
            "estimator_class": est_cls
        }
        print(f"[CV] {name}: SMAPE={mean_smape:.4f}")

    # 최적 모델 선택
    best_name = min(results, key=lambda k: results[k]["cv_smape"])
    best_info = results[best_name]

    # 최적 모델을 전체 학습 데이터로 재학습
    best_est = best_info["estimator_class"](**best_info["params"])
    best_est.fit(X_train, y_train)

    return {
        "model_name": best_name,
        "smape": best_info["cv_smape"],
        "estimator": best_est,
        "params": best_info["params"],
        "color": best_info["color"]
    }

# ==== 전체 타입별 학습/선택 ====
target_col = "power_consumption"
all_results = {}
best_by_type = {}

for btype_name, df in df_dict.items():
    X_train = df.drop(columns=[target_col])
    y_train = df[target_col]
    best = train_and_select_without_tuning(models, X_train, y_train)
    all_results[btype_name] = {best["model_name"]: best}  
    best_by_type[btype_name] = best
    print(f"[SELECT] {btype_name}: {best['model_name']} (SMAPE={best['smape']:.4f})")


In [ ]:
import pickle
import os

save_dir = "../models/ml/"
os.makedirs(save_dir, exist_ok=True)

for btype, info in best_by_type.items():
    model = info["estimator"]
    filename = os.path.join(save_dir, f"{btype}_{info['model_name']}.pkl")
    with open(filename, "wb") as f:
        pickle.dump(model, f)
    print(f"Saved: {filename}")

In [55]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import os 
import pickle

test_path = "C:/Project/DACON_Project/Electricity_prediction/data/origin/test.csv"
train_path = "C:/Project/DACON_Project/Electricity_prediction/data/processed"
building_path = "C:/Project/DACON_Project/Electricity_prediction/data/origin/building_info.csv"
models = "C:/Project/DACON_Project/Electricity_prediction/models/ml"
sample_path = "C:/Project/DACON_Project/Electricity_prediction/data/origin/sample_submission.csv"
test_dict = {}
answer_list = []
change_name = ['hotel', 'commercial', 'hospital', 'school', 'etc', 'apart', 'research', 'store', 'idc','public']
ko2en_dict = {
 '건물번호': 'b_num',
 '일시': 'date',
 '기온(°C)': 'tmp',
 '강수량(mm)': 'rain',
 '풍속(m/s)': 'wind',
 '습도(%)': 'hum',
 '일조(hr)': 'sunshine',
 '일사(MJ/m2)': 'solar',
 '전력소비량(kWh)': 'power_consumption',
 '건물유형': 'b_type',
 '연면적(m2)': 'total_area',
 '냉방면적(m2)': 'cooling_area',
 '태양광용량(kW)': 'solar_capacity',
 'ESS저장용량(kWh)': 'ess_capacity',
 'PCS용량(kW)': 'pcs_capacity',
}

def rename_dataframe_columns(df, mapping_dict):
    return df.rename(columns=mapping_dict).copy()

def minmax_scale(df: pd.DataFrame, exclude_cols):
    '''MinMax Scalering 적용'''
    target_cols = [i for i in df.columns if i not in exclude_cols]

    scaler = MinMaxScaler()
    df[target_cols] = scaler.fit_transform(df[target_cols])

    return df 

test = pd.read_csv(test_path, encoding='utf-8')
building = pd.read_csv(building_path, encoding='utf-8')

test_df = rename_dataframe_columns(test, ko2en_dict)
test_df['datetime'] = pd.to_datetime(test['일시'], format='%Y%m%d %H')

test_df['weekday'] = test_df['datetime'].dt.weekday 
test_df['time'] = test_df['datetime'].dt.hour

building_info_df = rename_dataframe_columns(building, ko2en_dict)
merge_df = pd.merge(test_df, building_info_df, on='b_num', how='left')


for i, tp in enumerate(merge_df['b_type'].unique()):
    name = change_name[i]
    test_dict[name] = merge_df[merge_df['b_type'] == tp].reset_index(drop=True)
    path = f"{train_path}/{name}_train.csv"
    train_df = pd.read_csv(path, encoding='utf-8')
    cols = list(train_df.columns)

    if "weekday" in cols:
        test_dict[name]['weekday'] = test_dict[name]['weekday'].apply(lambda x: 0 if x < 5 else 1)
    x_test = test_dict[name][cols]

    exclude_cols = ['weekday'] if 'weekday' in cols else []

    x_test = test_dict[name][cols].copy()
    x_test = minmax_scale(x_test, exclude_cols)

    if [f for f in os.listdir(models) if name in f]:
        model_path = os.path.join(models,[f for f in os.listdir(models) if name in f][0])
    else:
        print(f"There isn't the {name} model")

    with open(model_path, 'rb') as f:
        model = pickle.load(f)

    y_pred = model.predict(x_test)
    answer_list.append(y_pred)
    print(f"{name} 예측 완료, 결과 샘플: {y_pred[:5]}")

submission = pd.read_csv('C:/Project/DACON_Project/Electricity_prediction/data/origin/sample_submission.csv')
submission['answer'] = answer_list
submission.to_csv(f'C:/Project/DACON_Project/Electricity_prediction/result/baseline_submission_ML.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Project/DACON_Project/Electricity_prediction/data/processed/hotel_train.csv'

### 1. 학습 데이터 셋 구축

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import make_scorer
import pandas as pd
import numpy as np
import os 
import pickle

train_path = "../data/origin/train.csv"
test_path = "../data/origin/test.csv"
building_path = "../data/origin/building_info.csv"
submission_path = "../data/origin/sample_submission.csv"

ko2en_dict = {
 '건물번호': 'b_num',
 '일시': 'date',
 '기온(°C)': 'tmp',
 '강수량(mm)': 'rain',
 '풍속(m/s)': 'wind',
 '습도(%)': 'hum',
 '일조(hr)': 'sunshine',
 '일사(MJ/m2)': 'solar',
 '전력소비량(kWh)': 'power_consumption',
 '건물유형': 'b_type',
 '연면적(m2)': 'total_area',
 '냉방면적(m2)': 'cooling_area',
 '태양광용량(kW)': 'solar_capacity',
 'ESS저장용량(kWh)': 'ess_capacity',
 'PCS용량(kW)': 'pcs_capacity',
}

change_name = ['hotel', 'commercial', 'hospital', 'school', 'etc', 'apart', 'research', 'store', 'idc','public']

train = pd.read_csv(train_path, encoding='utf-8')
test = pd.read_csv(test_path, encoding='utf-8')
building = pd.read_csv(building_path, encoding='utf-8')

In [57]:
def rename_dataframe_columns(df, mapping_dict):
    return df.rename(columns=mapping_dict).copy()

def add_time(df):
    df['datetime'] = pd.to_datetime(df['date'], format='%Y%m%d %H')
    df['weekday'] = df['datetime'].dt.weekday
    df['time'] = df['datetime'].dt.hour   
    return df

In [58]:
train_df = rename_dataframe_columns(train, ko2en_dict)
test_df = rename_dataframe_columns(test, ko2en_dict)
building_info_df = rename_dataframe_columns(building, ko2en_dict)

train_df = add_time(train_df)
test_df = add_time(test_df)

train_merge = pd.merge(train_df, building_info_df, on='b_num', how='left')
test_merge = pd.merge(test_df, building_info_df, on='b_num', how='left')

btypes = list(building_info_df['b_type'].unique())
type_map = {bt: change_name[i] for i, bt in enumerate(btypes)}
train_merge['b_type'] = train_merge['b_type'].apply(lambda x : type_map[x])
test_merge['b_type'] = test_merge['b_type'].apply(lambda x : type_map[x])

display(train_merge.head())
print(list(train_merge.columns))
display(test_merge.head())
print(list(test_merge.columns))

,num_date_time,b_num,date,tmp,rain,wind,hum,sunshine,solar,power_consumption,datetime,weekday,time,b_type,total_area,cooling_area,solar_capacity,ess_capacity,pcs_capacity
0,1_20240601 00,1,20240601 00,18.3,0.0,2.6,82.0,0.0,0.0,5794.80,2024-06-01 00:00:00,5,0,hotel,82912.71,77586.0,-,-,-
1,1_20240601 01,1,20240601 01,18.3,0.0,2.7,82.0,0.0,0.0,5591.85,2024-06-01 01:00:00,5,1,hotel,82912.71,77586.0,-,-,-
2,1_20240601 02,1,20240601 02,18.1,0.0,2.6,80.0,0.0,0.0,5338.17,2024-06-01 02:00:00,5,2,hotel,82912.71,77586.0,-,-,-
3,1_20240601 03,1,20240601 03,18.0,0.0,2.6,81.0,0.0,0.0,4554.42,2024-06-01 03:00:00,5,3,hotel,82912.71,77586.0,-,-,-
4,1_20240601 04,1,20240601 04,17.8,0.0,1.3,81.0,0.0,0.0,3602.25,2024-06-01 04:00:00,5,4,hotel,82912.71,77586.0,-,-,-


['num_date_time', 'b_num', 'date', 'tmp', 'rain', 'wind', 'hum', 'sunshine', 'solar', 'power_consumption', 'datetime', 'weekday', 'time', 'b_type', 'total_area', 'cooling_area', 'solar_capacity', 'ess_capacity', 'pcs_capacity']


,num_date_time,b_num,date,tmp,rain,wind,hum,datetime,weekday,time,b_type,total_area,cooling_area,solar_capacity,ess_capacity,pcs_capacity
0,1_20240825 00,1,20240825 00,26.5,0.0,0.7,80.0,2024-08-25 00:00:00,6,0,hotel,82912.71,77586.0,-,-,-
1,1_20240825 01,1,20240825 01,26.1,0.0,0.0,80.0,2024-08-25 01:00:00,6,1,hotel,82912.71,77586.0,-,-,-
2,1_20240825 02,1,20240825 02,25.9,0.0,0.3,83.0,2024-08-25 02:00:00,6,2,hotel,82912.71,77586.0,-,-,-
3,1_20240825 03,1,20240825 03,25.7,0.0,1.1,83.0,2024-08-25 03:00:00,6,3,hotel,82912.71,77586.0,-,-,-
4,1_20240825 04,1,20240825 04,25.5,0.0,1.0,86.0,2024-08-25 04:00:00,6,4,hotel,82912.71,77586.0,-,-,-


['num_date_time', 'b_num', 'date', 'tmp', 'rain', 'wind', 'hum', 'datetime', 'weekday', 'time', 'b_type', 'total_area', 'cooling_area', 'solar_capacity', 'ess_capacity', 'pcs_capacity']


In [59]:
def is_drop(df, col):
    '''col 전체가 결측치면 True 하나라도 참값이 있다면 False'''
    if df[col].isna().all():
        return True
    else:
        return False
    
def convert_day(df, threshold=0.018):
    '''주말의 평균 전력사용량과 평일의 전력사용량의 비율 차이가 0.018 이상 차이나면 범주화'''
    gb = df.groupby(['weekday']).mean(numeric_only=True)

    workday = (sum(gb['power_consumption'].iloc[0:5])/5)/sum(gb['power_consumption']) 
    holiday = (sum(gb['power_consumption'].iloc[5:])/2)/sum(gb['power_consumption'])
    if workday - holiday > threshold:
        return True
    else:
        return False
    
def minmax_scale(df: pd.DataFrame, exclude_cols, scaler):
    '''MinMax Scalering 적용'''
    target_cols = [i for i in df.columns if i not in exclude_cols]

    df[target_cols] = scaler.fit_transform(df[target_cols])

    return df

def smape(y_true, y_pred):
    return 100 * np.mean(
        2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-9)
    )
smape_scorer = make_scorer(lambda y_true, y_pred: -smape(y_true, y_pred))

default_params = {
    "XGB": {
        "n_estimators": 500,
        "learning_rate": 0.1,
        "max_depth": 7,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "random_state": 42,
        "n_jobs": -1,
        "verbosity": 0  
    },
    "LGBM": {
        "n_estimators": 500,
        "learning_rate": 0.1,
        "max_depth": -1,
        "num_leaves": 63,
        "subsample": 0.8,
        "random_state": 42,
        "n_jobs": -1,
        "verbose": -1   
    },
    "RF": {
        "n_estimators": 500,
        "max_depth": None,
        "min_samples_split": 2,
        "min_samples_leaf": 1,
        "max_features": "sqrt",
        "random_state": 42,
        "n_jobs": -1
    }
}


In [60]:
scaler = MinMaxScaler()
result = {}

for tp in change_name:
    exclude_list = ['time']
    drop_cols = ['num_date_time', 'b_num', 'date','datetime', 'b_type', 'total_area','rain', 'sunshine', 'solar']

    train_type_df = train_merge[train_merge['b_type'] == tp].reset_index(drop=True)
    test_type_df = test_merge[test_merge['b_type'] == tp].reset_index(drop=True)

    for col in train_type_df.columns:
        if is_drop(train_type_df, col) and col not in drop_cols:
            drop_cols.append(col)
    
    if convert_day(train_type_df):
        train_type_df['weekday'] = train_type_df['weekday'].apply(lambda x: 0 if x < 5 else 1)
        test_type_df['weekday'] = test_type_df['weekday'].apply(lambda x: 0 if x < 5 else 1)
        exclude_list.append('weekday')
    else:
        drop_cols.append('weekday')

    train_type_df.drop(drop_cols, axis=1, inplace=True)

    train_type_df = train_type_df.replace("-", 0)
    test_type_df = test_type_df.replace("-", 0)

    y_train = train_type_df['power_consumption'].copy()
    X_train = train_type_df.drop(columns=['power_consumption']).copy()
    X_test = test_type_df[X_train.columns]

    exclude_train = exclude_list + ['power_consumption']  
    exclude_test = exclude_list                           

    X_train = minmax_scale(X_train, exclude_train, scaler)
    X_test = minmax_scale(X_test, exclude_test, scaler)

    X_tr, X_vr, y_tr, y_vr = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    model_builders = {
        "XGB": lambda: XGBRegressor(**default_params["XGB"]),
        "LGBM": lambda: LGBMRegressor(**default_params["LGBM"]),
        "RF": lambda: RandomForestRegressor(**default_params["RF"])
    }

    best_model = None
    best_score = float('inf')

    # 세 가지 모델 학습 및 검증
    for name, build_model in model_builders.items():
        model = build_model()
        model.fit(X_tr, y_tr)
        val_pred = model.predict(X_vr)
        score = smape(y_vr, val_pred)
        print(f"Model: {name}, Validation SMAPE: {score:.4f}")
        if score < best_score:
            best_score = score
            best_model = model
            best_name = name

    print(f"Best model: {best_name} with SMAPE {best_score:.4f}")

    # 최적 모델로 전체 train 데이터 재학습
    best_model.fit(X_train, y_train)

    # 테스트 데이터 예측
    test_pred = best_model.predict(X_test)
    print(f"Test prediction sample: {test_pred[:5]}")
    result[tp] = test_pred

C:\Users\gd980\AppData\Local\Temp\ipykernel_20376\2244402278.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_type_df = train_type_df.replace("-", 0)
C:\Users\gd980\AppData\Local\Temp\ipykernel_20376\2244402278.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_type_df = test_type_df.replace("-", 0)
C:\Users\gd980\AppData\Local\Temp\ipykernel_20376\1114969982.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

Model: XGB, Validation SMAPE: 11.2057
Model: LGBM, Validation SMAPE: 10.2986
Model: RF, Validation SMAPE: 13.3219
Best model: LGBM with SMAPE 10.2986
Test prediction sample: [5485.49377852 5460.27561434 4958.53495842 4497.01423325 4521.82998641]


C:\Users\gd980\AppData\Local\Temp\ipykernel_20376\2244402278.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_type_df = train_type_df.replace("-", 0)
C:\Users\gd980\AppData\Local\Temp\ipykernel_20376\2244402278.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_type_df = test_type_df.replace("-", 0)
C:\Users\gd980\AppData\Local\Temp\ipykernel_20376\1114969982.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

Model: XGB, Validation SMAPE: 4.1838
Model: LGBM, Validation SMAPE: 4.0417
Model: RF, Validation SMAPE: 4.4043
Best model: LGBM with SMAPE 4.0417
Test prediction sample: [1381.38355613 1128.06436044 1073.78489879 1004.59345147 1008.58904713]


C:\Users\gd980\AppData\Local\Temp\ipykernel_20376\1114969982.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[target_cols] = scaler.fit_transform(df[target_cols])


...

Model: XGB, Validation SMAPE: 9.8563
Model: LGBM, Validation SMAPE: 9.3252
Model: RF, Validation SMAPE: 10.9400
Best model: LGBM with SMAPE 9.3252
Test prediction sample: [533.67283928 481.65681941 477.57496527 478.14738864 460.304802  ]


C:\Users\gd980\AppData\Local\Temp\ipykernel_20376\2244402278.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_type_df = train_type_df.replace("-", 0)
C:\Users\gd980\AppData\Local\Temp\ipykernel_20376\2244402278.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_type_df = test_type_df.replace("-", 0)
C:\Users\gd980\AppData\Local\Temp\ipykernel_20376\1114969982.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

Model: XGB, Validation SMAPE: 2.5360
Model: LGBM, Validation SMAPE: 2.4235
Model: RF, Validation SMAPE: 3.1192
Best model: LGBM with SMAPE 2.4235
Test prediction sample: [9652.73833142 9628.96373071 9584.77393914 9615.2300747  9629.4566777 ]


C:\Users\gd980\AppData\Local\Temp\ipykernel_20376\1114969982.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[target_cols] = scaler.fit_transform(df[target_cols])


Model: XGB, Validation SMAPE: 8.4045
Model: LGBM, Validation SMAPE: 8.3330
Model: RF, Validation SMAPE: 7.8274
Best model: RF with SMAPE 7.8274
Test prediction sample: [1662.84032 1304.08016 1117.0609  1062.84748 1091.74976]


In [61]:
y_out = np.zeros(len(test_merge))
for tp in change_name:
    idx = test_merge.index[test_merge['b_type'] == tp].tolist()
    y_out[idx] = result[tp]

submission = pd.read_csv(submission_path)
submission['answer'] = y_out
submission.to_csv("ml.csv", index=False)
print("저장 완료: baseline_submission.csv")

저장 완료: baseline_submission.csv
